In [4]:
pip install transformers


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install "sglang[all]"


Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install flashinfer -i https://flashinfer.ai/whl/cu121/torch2.4/


Looking in indexes: https://flashinfer.ai/whl/cu121/torch2.4/
Note: you may need to restart the kernel to use updated packages.


In [1]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:30000/v1", api_key="None")

response = client.chat.completions.create(
    model="Qwen2-VL-72B-Instruct-AWQ",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What is in this image?",
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://main-cdn.sbermegamarket.ru/hlr-system/-24/181/909/262/012/24/100032273383b0.jpg"
                    },
                },
            ],
        }
    ],
    max_tokens=300,
)

print(response.choices[0].message.content)

The image shows a black leather handbag with a structured design. The bag has a top handle made of the same black leather, and there is a metallic clasp on the front flap. The bag appears to be made of high-quality leather with visible grain, giving it a textured look. The overall design is elegant and minimalist, with clean lines and a sleek finish.


Товар:
{item[2]['товар']}

Характеристика товара:
{item[4]["характеристики"]}

Ссылка на изображение товара:
{item[1]['image_link']}

Запрос:
{item[3]['запрос']}

In [ ]:
1.
Ты помощник поиска в маркетплейсе.
Тебе необходимо определить, соответствует ли товар, его фото и характеристики поисковому запросу.

Классы запросов:
* relevant - товары, которые можно купить по указанному запросу.
* irrelevant - товары, которые было бы странно увидеть по указанному запросу.

Определи релевантность товара запросу. Отвечай только "relevant" или "irrelevant". Не пиши объяснений и ничего больше.

Товар:
{item[2]['товар']}


Подумай, проанализируй товар (только ничего не пиши). Выдели самые его основные и ключевые характеристики (ничего не пиши, думай).
Так же проанализируй изображение товара (только ничего не пиши).

Запрос:
{item[3]['запрос']}


Accuracy: 86.25%
Accuracy for relevant: 96.39%
Accuracy for irrelevant: 73.53% 


2. Ты главный специалист команды поиска в маркетплейсе. Ты должен определить, подходит ли товар и его фото из поисковой выдачи запросу или нет.
Твои варианты ответов:
* relevant - товары, которые можно купить по указанному запросу.
* irrelevant - товары, которые было бы странно увидеть по указанному запросу.

Определи, релевантен ли товар и его фото поисковому запросу. Отвечай только "relevant" или "irrelevant".
Товар:
{item[2]['товар']}
Запрос:
{item[3]['запрос']}


Accuracy: 85.00%
Accuracy for relevant: 94.88%
Accuracy for irrelevant: 73.53% 


3.
Ты ведущий эксперт команды поиска в маркетплейсе. Твоя задача — определить, соответствует ли товар и его изображение запросу из поисковой выдачи. При этом анализируй как текстовое описание товара, так и фото. Обрати внимание на изображение: оно должно точно отражать товар, который подходит запросу, включая его внешний вид, форму, цвет, и другие ключевые визуальные характеристики.

Варианты ответов:

	•	relevant — товар, который подходит под указанный запрос, а его фото соответствует описанию и намерению запроса.
	•	irrelevant — товар, который не соответствует запросу или чье изображение вводит в заблуждение.

Оцени соответствие товара и его изображения запросу. Отвечай только ‘relevant’ или ‘irrelevant’.
Товар: {item[2][‘товар’]}
Запрос: {item[3][‘запрос’]}


Accuracy: 84.00%
Accuracy for relevant: 92.77%
Accuracy for irrelevant: 82.35% 


4.
Ты ведущий эксперт команды поиска в маркетплейсе. Твоя задача — определить, соответствует ли товар и его изображение запросу из поисковой выдачи, оценивая их с точки зрения пользователя.

Анализируй:
	1.	Название товара — соответствует ли оно запросу, понятно ли, что это за товар, и заинтересует ли оно пользователя.
	2.	Фото товара — отражает ли оно товар, который пользователь ожидает увидеть по запросу, ясно ли на изображении, что это именно искомый предмет, и привлекательно ли оно выглядит.

Варианты ответов:
	•	relevant — товар и его изображение соответствуют запросу и удовлетворяют ожидания пользователя.
	•	irrelevant — товар или его изображение не соответствуют запросу и могут разочаровать пользователя.

Оцени соответствие товара и его изображения запросу. Отвечай только ‘relevant’ или ‘irrelevant’.
Товар: {item[2][‘товар’]}
Запрос: {item[3][‘запрос’]}


Accuracy: 85.00%
Accuracy for relevant: 94.28%
Accuracy for irrelevant: 79.41% 



# Qwen2-VL-72B-Instruct-AWQ на 400 запросах с фото

In [13]:
%%time

import openai
import json

client = openai.Client(base_url="http://127.0.0.1:30000/v1", api_key="None")

# загружаю файлы (с примерами и правильными ответами)
with open('files/RI разметка/2 итерация/items_400_images (2).jsonl', 'r') as f:
    items_data = [json.loads(line) for line in f]
with open('files/RI разметка/2 итерация/data_vector_gt_400_orig_images (2).txt', 'r') as f:
    ground_truth = f.read().strip().splitlines()

results = []

for i, item in enumerate(items_data):
    system = f"""\
Ты ведущий эксперт команды поиска в маркетплейсе. Твоя задача — определить, соответствует ли товар и его изображение запросу из поисковой выдачи. При этом анализируй как текстовое описание товара, так и фото. Обрати внимание на изображение: оно должно точно отражать товар, который подходит запросу, включая его внешний вид, форму, цвет, и другие ключевые визуальные характеристики.

Варианты ответов:
	•	relevant — товар, который подходит под указанный запрос, а его фото соответствует описанию и намерению запроса.
	•	irrelevant — товар, который не соответствует запросу или чье изображение вводит в заблуждение.


Примеры нерелевантных товаров к запросам:
1:
Товар: часы с ремешком
Запрос: ремешок для часов
Оценка релевантности: irrelevant
2:
Товар: футболка M
Запрос: футболка размер s
Оценка релевантности: irrelevant
3:
Товар: чехол на iPhone 15
Запрос: айфон 15
Оценка релевантности: irrelevant
"""

    system_query = f"""\
Оцени соответствие товара и его изображения запросу. Отвечай только ‘relevant’ или ‘irrelevant’.

Товар: {item[2]['товар']}
Запрос: {item[3]['запрос']}
Оценка релевантности:
"""

    messages = [
        {"role": "user", "content": [{"type": "text", "text": system}]}
    ]

    # ссылка на изображение
    if item[1].get('image_link'):
        messages[0]["content"].append(
            {"type": "image_url", "image_url": {"url": item[1]['image_link']}}
        )

    messages[0]["content"].append({"type": "text", "text": system_query})

    try:
        response = client.chat.completions.create(
            model="Qwen2-VL-72B-Instruct-AWQ",
            messages=messages,
            # top_p=0.1
            # temperature = 0.01
            # max_tokens=1 # плохо сработало
        )

        prediction = response.choices[0].message.content.strip().lower()

        # сравниваю предсказание с правильным ответом, исключая #N/A
        ground_truth_label = ground_truth[i].strip().lower()
        if ground_truth_label != '#n/a':
            print(i, " ", ground_truth_label, " ", prediction)
            is_correct = prediction == ground_truth_label
            results.append({
                "ID": item[0]['id'],
                "запрос": item[3]['запрос'],
                "товар": item[2]['товар'],
                "предсказание": prediction,
                "реальность": ground_truth_label,
                "is_correct": is_correct
            })
    except Exception as e:
        print(f"Error processing item {i}: {e}")

0   relevant   relevant
1   relevant   relevant
2   relevant   relevant
3   relevant   relevant
4   relevant   relevant
5   relevant   relevant
6   relevant   relevant
7   relevant   relevant
8   relevant   relevant
9   relevant   relevant
10   relevant   relevant
11   relevant   relevant
12   relevant   relevant
13   relevant   relevant
14   relevant   relevant
15   relevant   relevant
16   relevant   relevant
17   relevant   relevant
18   relevant   relevant
19   relevant   relevant
20   relevant   relevant
21   relevant   relevant
22   relevant   relevant
23   relevant   irrelevant
24   relevant   irrelevant
25   relevant   relevant
26   relevant   relevant
27   relevant   relevant
28   no_chars   relevant
29   relevant   relevant
30   relevant   relevant
31   relevant   relevant
32   relevant   relevant
33   relevant   relevant
34   relevant   relevant
35   relevant   relevant
36   relevant   relevant
37   relevant   relevant
38   relevant   relevant
39   relevant   relevant
40   r

In [14]:
# общая Accuracy
count_correct = sum(1 for item in results if item['is_correct'] == True)
total_predictions = len(results)
accuracy = count_correct / total_predictions if total_predictions > 0 else 0
print(f"Accuracy: {accuracy:.2%}")

# Accuracy для relevant
relevant_correct = sum(1 for item in results if item['реальность'] == 'relevant' and item['is_correct'])
relevant_total = sum(1 for item in results if item['реальность'] == 'relevant')
relevant_accuracy = relevant_correct / relevant_total if relevant_total > 0 else 0
print(f"Accuracy for relevant: {relevant_accuracy:.2%}")

# Accuracy для irrelevant
irrelevant_correct = sum(1 for item in results if item['реальность'] == 'irrelevant' and item['is_correct'])
irrelevant_total = sum(1 for item in results if item['реальность'] == 'irrelevant')
irrelevant_accuracy = irrelevant_correct / irrelevant_total if irrelevant_total > 0 else 0
print(f"Accuracy for irrelevant: {irrelevant_accuracy:.2%} \n")

for item in results:
    if not item["is_correct"] and item["реальность"] == "irrelevant":
        print(item, "\n")

Accuracy: 85.50%
Accuracy for relevant: 94.58%
Accuracy for irrelevant: 82.35% 

{'ID': 219, 'запрос': 'xbox 360', 'товар': 'Игра SoulCalibur 5 (V) Русская Версия для Microsoft Xbox 360', 'предсказание': 'relevant', 'реальность': 'irrelevant', 'is_correct': False} 

{'ID': 248, 'запрос': 'hover h3 магнитола', 'товар': 'Переходная рамка для магнитолы Wide Media Great Wall Hover H3 2010-2014 9"', 'предсказание': 'relevant', 'реальность': 'irrelevant', 'is_correct': False} 

{'ID': 258, 'запрос': 'все для рыбалки', 'товар': 'Налобный светодиодный фонарь Double Light KX-1804', 'предсказание': 'relevant', 'реальность': 'irrelevant', 'is_correct': False} 

{'ID': 264, 'запрос': 'l7', 'товар': 'Чехол для ключа AA для LiXiang L7, L8, L9', 'предсказание': 'relevant', 'реальность': 'irrelevant', 'is_correct': False} 

{'ID': 272, 'запрос': 'iphone 7￼', 'товар': 'Чехол накладка для iPhone 7 с подкладкой из микрофибры / черный', 'предсказание': 'relevant', 'реальность': 'irrelevant', 'is_correct':

# Chain Of Thought

In [11]:
%%time

import openai
import json

client = openai.Client(base_url="http://127.0.0.1:30000/v1", api_key="None")

# загружаю файлы (с примерами и правильными ответами)
with open('files/RI разметка/2 итерация/items_400_images (2).jsonl', 'r') as f:
    items_data = [json.loads(line) for line in f]
with open('files/RI разметка/2 итерация/data_vector_gt_400_orig_images (2).txt', 'r') as f:
    ground_truth = f.read().strip().splitlines()

results = []

for i, item in enumerate(items_data):
    system = f"""\
Ты ведущий эксперт команды поиска в маркетплейсе. Твоя задача — определить, соответствует ли товар и его изображение запросу из поисковой выдачи. При этом анализируй как текстовое описание товара, так и фото.

Варианты ответов:
	•	relevant — товар, который подходит под указанный запрос, а его фото соответствует описанию и намерению запроса.
	•	irrelevant — товар, который не соответствует запросу или чье изображение вводит в заблуждение.

Step-by-step reasoning:
1. Проверяем, относится ли товар к категории, запрашиваемой пользователем.
2. Проверяем, есть ли в описании товара характеристики, соответствующие запросу.
3. Проверяем изображение: оно должно точно отражать товар, который подходит запросу, включая его внешний вид, форму, цвет, и другие ключевые визуальные характеристики.
4. Делаем вывод о релевантности.

Пример 1:
Запрос: "телефон с хорошей камерой"
Товар: "Смартфон Xiaomi 13 Pro, камера 200 МП, 256 ГБ памяти"
Step-by-step reasoning:
1. Выделяем главное слово в запросе (телефон) и товаре (Cмартфон). 
2. Проверяем, совпадают ли по смыслу главные слова в запросе и товаре: да, телефон и Смартфон это одно и тоже
2. Проверяем, упоминается ли камера в характеристиках. Ответ: Да, 200 МП.
3. Сравниваем характеристики камеры с запросом "хорошая камера". Ответ: Да, камера высокого качества.
Вывод: relevant

Пример 2:
Запрос: "чехол на iphone 15"
Товар: "Iphone 15, камера 200 МП"
Step-by-step reasoning:
1. Выделяем главное слово в запросе (чехол) и товаре (Iphone). 
2. Проверяем, совпадают ли по смыслу главные слова в запросе и товаре: нет
Вывод: irrelevant
"""

    system_query = f"""\
Оцени соответствие товара и его изображения запросу.

Товар: {item[2]['товар']}
Запрос: {item[3]['запрос']}
"""

    messages = [
        {"role": "user", "content": [{"type": "text", "text": system}]}
    ]

    # ссылка на изображение
    if item[1].get('image_link'):
        messages[0]["content"].append(
            {"type": "image_url", "image_url": {"url": item[1]['image_link']}}
        )

    messages[0]["content"].append({"type": "text", "text": system_query})

    try:
        response = client.chat.completions.create(
            model="Qwen2-VL-72B-Instruct-AWQ",
            messages=messages,
            # top_p=0.1
            # temperature = 0.01
            # max_tokens=1 # плохо сработало
        )

        prediction = response.choices[0].message.content.strip().lower()

        # сравниваю предсказание с правильным ответом, исключая #N/A
        ground_truth_label = ground_truth[i].strip().lower()
        if ground_truth_label != '#n/a':
            print(i, " ", ground_truth_label, " ", prediction)
            print()
            
            results.append({
                "ID": item[0]['id'],
                "запрос": item[3]['запрос'],
                "товар": item[2]['товар'],
                "предсказание": prediction,
                "реальность": ground_truth_label,
            })
    except Exception as e:
        print(f"Error processing item {i}: {e}")



0   relevant   step-by-step reasoning:

1. выделяем главное слово в запросе (зарядный) и товаре (usb кабель). проверяем, совпадают ли по смыслу главные слова в запросе и товаре: да, usb кабель может использоваться как зарядный провод.

2. проверяем, есть ли в описании товара характеристики, соответствующие запросу. ответ: да, usb кабель может использоваться для зарядки устройств.

3. проверяем изображение: оно должно точно отражать товар, который подходит запросу, включая его внешний вид, форму, цвет, и другие ключевые визуальные характеристики. ответ: да, изображение показывает usb кабель, который соответствует запросу.

вывод: relevant

Error processing item 1: Internal Server Error
2   relevant   step-by-step reasoning:
1. выделяем главное слово в запросе (дрель шуруповерт) и товаре (дрель-шуруповерт). 
2. проверяем, совпадают ли по смыслу главные слова в запросе и товаре: да, "дрель шуруповерт" и "дрель-шуруповерт" это одно и то же.
3. проверяем изображение: оно точно отражает това

In [12]:
final = results

In [13]:
for result in results:
    if "irrelevant" in result["предсказание"]:
        result["ПРЕДСКАЗАНИЕ ЭТО"] = "irrelevant"
    elif "relevant" in result["предсказание"]:
        result["ПРЕДСКАЗАНИЕ ЭТО"] = "relevant"
    else:
        result["ПРЕДСКАЗАНИЕ ЭТО"] = "Ошибка"

In [20]:
for result in results[:5]:
    print(result)
    print()

{'ID': 209, 'запрос': 'redb', 'товар': 'Сельдь Исландка кусочки в красном вине 115 г', 'предсказание': 'step-by-step reasoning:\n\n1. выделяем главное слово в запросе (red) и товаре (сельдь). \n2. проверяем, совпадают ли по смыслу главные слова в запросе и товаре: нет, "red" и "сельдь" не совпадают.\n3. проверяем, есть ли в описании товара характеристики, соответствующие запросу. в данном случае, "red" может отсылать к красному вине в описании товара.\n4. проверяем изображение: на изображении изображены кусочки сельди в красной жидкости, вероятно, это красное вино, указанное в описании товара.\n\nвывод: relevant', 'реальность': 'irrelevant', 'ПРЕДСКАЗАНИЕ ЭТО': 'relevant', 'is_correct': False}

{'ID': 210, 'запрос': 'готическая  бижутерия', 'товар': 'Чокер из бижутерного сплава 30 см Queen Fair Цепь Звенья', 'предсказание': 'step-by-step reasoning:\n1. проверяем, относится ли товар к категории, запрашиваемой пользователем. в запросе указано "готическая бижутерия", а товар - "чокер из б

In [15]:
for result in results:
    is_correct = result["ПРЕДСКАЗАНИЕ ЭТО"] == result["реальность"]
    result["is_correct"] = is_correct

### Релевантные

In [21]:
for result in results:
    if not result["is_correct"] and result["реальность"] == "relevant":
        print(result["ID"])
        print(f'ЗАПРОС: {result["запрос"]}')
        print(f'ТОВАР: {result["товар"]}')
        print(f'РЕАЛЬНОСТЬ: {result["реальность"]}')
        print(f'ПРЕДСКАЗАНИЕ: {result["ПРЕДСКАЗАНИЕ ЭТО"]}')
        print(f'ТЕКСТ: \n {result["предсказание"]}')
        print()

12
ЗАПРОС: кухонное полотенце
ТОВАР: Полотенце Русский Дом 35 х 60 см рогожка темно-зеленое
РЕАЛЬНОСТЬ: relevant
ПРЕДСКАЗАНИЕ: irrelevant
ТЕКСТ: 
 step-by-step reasoning:
1. выделяем главное слово в запросе (полотенце) и товаре (полотенце). 
2. проверяем, совпадают ли по смыслу главные слова в запросе и товаре: да, оба указывают на полотенце.
3. проверяем, есть ли в описании товара характеристики, соответствующие запросу "кухонное полотенце". ответ: нет, в описании не указано, что это кухонное полотенце.
4. проверяем изображение: на изображении показано полотенце, которое может быть кухонным. однако, без текстового описания, сложно сделать окончательный вывод.
вывод: irrelevant

20
ЗАПРОС: крижная полка навесная
ТОВАР: Полка навесная Olivia
РЕАЛЬНОСТЬ: relevant
ПРЕДСКАЗАНИЕ: irrelevant
ТЕКСТ: 
 step-by-step reasoning:
1. выделяем главное слово в запросе (полка) и товаре (полка): да, совпадают.
2. проверяем, совпадают ли по смыслу главные слова в запросе и товаре: да, полка и полка это 

### Нерелевантные

In [22]:
for result in results:
    if not result["is_correct"] and result["реальность"] == "irrelevant":
        print(result["ID"])
        print(f'ЗАПРОС: {result["запрос"]}')
        print(f'ТОВАР: {result["товар"]}')
        print(f'РЕАЛЬНОСТЬ: {result["реальность"]}')
        print(f'ПРЕДСКАЗАНИЕ: {result["ПРЕДСКАЗАНИЕ ЭТО"]}')
        print(f'ТЕКСТ: \n {result["предсказание"]}')
        print()

124
ЗАПРОС: матрас жесткий
ТОВАР: Ортопедический Матрас Darwin Breeze 90х190, беспружинный, 21 см высота, белый
РЕАЛЬНОСТЬ: irrelevant
ПРЕДСКАЗАНИЕ: relevant
ТЕКСТ: 
 step-by-step reasoning:

1. категория товара: ортопедический матрас.
2. запрос: "матрас жесткий".
3. описание товара упоминает жесткость: нет.
4. изображение товара: соответствует описанию, но не предоставляет информации о жесткости.

вывод: relevant

обоснование: товар соответствует запросу по категории (матрас). изображение точно отражает товар, но для полной уверенности в релевантности необходимо было бы уточнение о жесткости товара в его описании.

209
ЗАПРОС: redb
ТОВАР: Сельдь Исландка кусочки в красном вине 115 г
РЕАЛЬНОСТЬ: irrelevant
ПРЕДСКАЗАНИЕ: relevant
ТЕКСТ: 
 step-by-step reasoning:

1. выделяем главное слово в запросе (red) и товаре (сельдь). 
2. проверяем, совпадают ли по смыслу главные слова в запросе и товаре: нет, "red" и "сельдь" не совпадают.
3. проверяем, есть ли в описании товара характеристики, со

In [23]:
# общая Accuracy
count_correct = sum(1 for item in results if item['is_correct'] == True)
total_predictions = len(results)
accuracy = count_correct / total_predictions if total_predictions > 0 else 0
print(f"Accuracy: {accuracy:.2%}")

# Accuracy для relevant
relevant_correct = sum(1 for item in results if item['реальность'] == 'relevant' and item['is_correct'])
relevant_total = sum(1 for item in results if item['реальность'] == 'relevant')
relevant_accuracy = relevant_correct / relevant_total if relevant_total > 0 else 0
print(f"Accuracy for relevant: {relevant_accuracy:.2%}")

# Accuracy для irrelevant
irrelevant_correct = sum(1 for item in results if item['реальность'] == 'irrelevant' and item['is_correct'])
irrelevant_total = sum(1 for item in results if item['реальность'] == 'irrelevant')
irrelevant_accuracy = irrelevant_correct / irrelevant_total if irrelevant_total > 0 else 0
print(f"Accuracy for irrelevant: {irrelevant_accuracy:.2%} \n")


Accuracy: 82.71%
Accuracy for relevant: 92.15%
Accuracy for irrelevant: 73.53% 

